In [14]:
import pandas as pd

ratings = pd.read_csv('/home/supay/datos/rating.csv')
ratings = ratings[['user_id','anime_id','rating']]
movies = pd.read_csv('/home/supay/datos/anime.csv')
movies = movies[['anime_id','name']]

#ratings = ratings.sample(n = 10000)
ratings = ratings.head(1167)
ratings = pd.merge(movies, ratings)

#dif = ratings.anime_id.unique()
#print(len(dif))

,anime_id,name,user_id,rating
0,32281,Kimi no Na wa.,6253,10
1,32281,Kimi no Na wa.,9247,10
2,32281,Kimi no Na wa.,72855,8
3,5114,Fullmetal Alchemist: Brotherhood,26502,10
4,5114,Fullmetal Alchemist: Brotherhood,11628,-1


In [2]:
userRatings = ratings.pivot_table(index=['user_id'],columns=['name'],values='rating')

In [4]:
corrMatrix = userRatings.corr(method='pearson', min_periods=5)
corrMatrix.head(1167)

name,11eyes,11eyes: Momoiro Genmutan,Absolute Duo,Accel World,Accel World EX,Afro Samurai,Afro Samurai: Resurrection,Air Gear,Akame ga Kill!,Akaneiro ni Somaru Saka,...,Zero no Tsukaima,Zero no Tsukaima F,Zero no Tsukaima: Futatsuki no Kishi,Zero no Tsukaima: Princesses no Rondo,Zetsuen no Tempest,Zetsumetsu Kigu Shoujo: Amazing Twins,Zoku Sayonara Zetsubou Sensei,Zombie-Loan,_Summer,xxxHOLiC
name,,,,,,,,,,,,,,,,,,,,,
11eyes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11eyes: Momoiro Genmutan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Absolute Duo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Accel World,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Accel World EX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Zetsumetsu Kigu Shoujo: Amazing Twins,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Zoku Sayonara Zetsubou Sensei,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Zombie-Loan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
myRatings = userRatings.loc[1].dropna()
myRatings.head(10)

name
11eyes                     -1.0
11eyes: Momoiro Genmutan   -1.0
Absolute Duo               -1.0
Accel World                -1.0
Akame ga Kill!             -1.0
Akaneiro ni Somaru Saka    -1.0
Akuma no Riddle            -1.0
Amagi Brilliant Park       -1.0
Angel Beats!               -1.0
Angel Beats! Specials      -1.0
Name: 1, dtype: float64

In [6]:
simCandidates = pd.Series()
print(simCandidates)
for i in range(0, len(myRatings.index)):
    print ("Añadiendo pelis similares a " + myRatings.index[i] + "...")
    # Recuperar las pelis similares a las calificadas
    sims = corrMatrix[myRatings.index[i]].dropna()
    # Escalar la similaridad multiplicando por la calificación de la persona
    sims = sims.map(lambda x: x * myRatings[i])
    # Añadir el puntaje a la lista de candidatos similares
    simCandidates = simCandidates.append(sims)
    
#Mirar los resultados:
#print ("ordenando...")
simCandidates.sort_values(inplace = True, ascending = False)
print (simCandidates.head(10))

<ipython-input-6-4a448cecd77e>:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  simCandidates = pd.Series()


Series([], dtype: float64)
Añadiendo pelis similares a 11eyes...
Añadiendo pelis similares a 11eyes: Momoiro Genmutan...
Añadiendo pelis similares a Absolute Duo...
Añadiendo pelis similares a Accel World...
Añadiendo pelis similares a Akame ga Kill!...
Añadiendo pelis similares a Akaneiro ni Somaru Saka...
Añadiendo pelis similares a Akuma no Riddle...
Añadiendo pelis similares a Amagi Brilliant Park...
Añadiendo pelis similares a Angel Beats!...
Añadiendo pelis similares a Angel Beats! Specials...
Añadiendo pelis similares a Angel Beats!: Another Epilogue...
Añadiendo pelis similares a Ao no Exorcist...
Añadiendo pelis similares a Ao no Exorcist Movie...
Añadiendo pelis similares a Ao no Exorcist: Kuro no Iede...
Añadiendo pelis similares a Asu no Yoichi!...
Añadiendo pelis similares a Black Bullet...
Añadiendo pelis similares a Blade &amp; Soul...
Añadiendo pelis similares a Boku wa Tomodachi ga Sukunai...
Añadiendo pelis similares a Boku wa Tomodachi ga Sukunai Episode 0...
Añadien

In [7]:
simCandidates = simCandidates.groupby(simCandidates.index).sum()

In [8]:
simCandidates.sort_values(inplace = True, ascending = False)
simCandidates.head(10)

Sword Art Online            9.848296
Log Horizon                 0.517042
Naruto                     -1.000000
Kami nomi zo Shiru Sekai   -1.000000
dtype: float64

In [9]:
#print(myRatings)
filteredSims = simCandidates.drop(myRatings.index,errors='ignore')
filteredSims.head(10)

Series([], dtype: float64)